In [347]:
from scipy.stats import expon
from scipy import special
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import random
from scipy.stats import t

def CrawfordTest(case, controls):
    """ Implementation of Crawford and Howell (1998; The Clinical Neuropsychologist) test to compare an individual to a sample.

    Args:
      case : score of the individual
      controls : scores of from a sample group

    Return:
      the one-tail probability associated to the score 'case' compared to the scores in the list 'controls'
    """
    if np.std(controls,ddof=1) == 0:
        tobs = case - np.mean(controls)
    else:
        tobs = (case - np.mean(controls)) / np.std(controls, ddof=1)
        
    if tobs >= 0:
        return (1 - t.cdf(tobs, len(controls) - 1))
    else:
        return t.cdf(tobs, len(controls) - 1)

#need to calculate real percentages off of this info instead of what we are already using
#what percentile does GPT answer in for each one?

def GenerateDistributions(df):
    for index, row in df.iterrows():
        mean = row['Humans Continuous']
        std = row['SD']
        case = row['GPT Continuous Temp 0']
        
        if std == 0:
            min_in_standard_domain = (min_value - mean)
            max_in_standard_domain = (max_value - mean)
        else:
            min_in_standard_domain = (min_value - mean) / std
            max_in_standard_domain = (max_value - mean) / std
    
        min_in_erf_domain = special.erf(min_in_standard_domain)
        max_in_erf_domain = special.erf(max_in_standard_domain)
    
        random_uniform_data = np.random.uniform(min_in_erf_domain, max_in_erf_domain, 10000)
        random_gaussianized_data = (special.erfinv(random_uniform_data) * std) + mean
    
        random_gaussianized_data = np.around(random_gaussianized_data,0)
    
        df.at[index, 'Distribution'] = random_gaussianized_data
    
        CrawfordTest(case, random_gaussianized_data)
        
#         fig, axes = plt.subplots(1, 1, figsize=(6, 3))
#         axes.hist(random_gaussianized_data, 5, range=(0,4))
#         axes.set_title(row[0])
#         plt.axvline(x=case, color='r')
# #     plt.axvline(x=row['GPT Continuous Temp .5'], color='y')
# #     plt.axvline(x=row['GPT Continuous Temp 1'], color='c')
# #    plt.savefig(row[0]+'.png')
#         plt.show()
    
        all_pairs_and_pvalues.update({row[0]:CrawfordTest(case, random_gaussianized_data)})
    
        if CrawfordTest(case, random_gaussianized_data) < p_value:
            significant_pairs.update({row[0]:CrawfordTest(case, random_gaussianized_data)})
            continue
        else:
            pass
    
        print(row[0], CrawfordTest(case, random_gaussianized_data))

# #print(all_pairs_and_pvalues)
# all_words_and_measures_short

In [293]:
#import and create cleaned stimuli strings which can be copied directly into GPT if desired
#find difference score between human ratings (Graves 2013) and GPTv3.5 (March 14)

home_direct = 'C:/Users/Nick/Downloads/meat-kangaroo-main/'

all_words_and_measures = pd.read_csv(home_direct + 'graves_with_vecs.csv', skipinitialspace=True, )
all_words_and_measures = all_words_and_measures.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])

all_words_and_measures['Humans Binary'] = np.where(
    all_words_and_measures['Humans Continuous'] >= 2.5, 'Makes Sense', np.where(
    all_words_and_measures['Humans Continuous'] <= 1.5, 'Nonsense', 'Ambiguous'))

#all_words_and_measures['Stim'] = 'the ' + all_words_and_measures['Stim'].str.replace('-', ' ') + '.'
all_words_and_measures['Difference Score'] = abs(all_words_and_measures['Humans Continuous'] - all_words_and_measures['GPT Continuous Temp 0'])

all_words_and_measures_short = all_words_and_measures.head(50)

# all_words_and_measures.to_excel('cleaned_words.xlsx')

In [348]:
p_value = .05

min_value = 0
max_value = 4
all_pairs_and_pvalues = {}
significant_pairs = {}

all_words_and_measures_short['Distribution'] = ''

GenerateDistributions(all_words_and_measures)

print('#########')
print(significant_pairs)
print('#############')
print('Number of significant pairs: ', len(significant_pairs))

all_words_and_measures

C:\Users\Nick\AppData\Local\Temp\ipykernel_16980\3482190932.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_words_and_measures_short['Distribution'] = ''


the dress summer. 0.1557913188879987
the pig truck. 0.4828521790770307
the heart guy. 0.1029050305082031
the helmet mirror. 0.3163179745835145
the field square. 0.26983812973132026
the pen pig. 0.17284396865659124
the bike barn. 0.319988997310549
the flag rainbow. 0.05158463376766198
the juice pickle. 0.09804996277070188
the beach summer. 0.2859499806518444
the dime bag. 0.17240845539572575
the sword cane. 0.11209463966338151
the sky rocket. 0.2599377705737067
the tree snow. 0.21466215380692266
the kitchen restaurant. 0.3055743969938829
the pig pet. 0.18463239094077888
the leg bird. 0.09145803652524842
the ocean swimming. 0.10910104687887034
the rocket guy. 0.3061937076233823
the door barn. 0.23655350893739935
the oven home. 0.09098535453323775
the toe nail. 0.06907467145826414
the skin banana. 0.06954462800921157
the winter wolf. 0.16216679935505596
the book jacket. 0.21755088721975097
the rock pet. 0.3368042536803828
the clothes police. 0.31686449453704646
the book baby. 0.1407762915

the cheese cottage. 0.06378956007376146
the salad potato. 0.1078573054885299
the frog soup. 0.3846022425500041
the milk nut. 0.052798977990592966
the palace window. 0.3904028165440088
the stream bed. 0.09730646052606917
the tape duck. 0.07187323615160601
the gorilla baby. 0.4859704740637689
the log cabin. 0.13457732557494786
the house flea. 0.24925587735152266
the golf holiday. 0.16738103009099348
the pup tent. 0.09121655690145558
the beach chair. 0.10509365817463501
the bed face. 0.08852634875519849
the bird leg. 0.10929281620801043
the water roach. 0.4836298911690461
the rocket ship. 0.3033871231895464
the sheep pen. 0.19646121021021568
the leaf picture. 0.180492995777215
the cake flour. 0.42000678835488237
the sugar water. 0.28740668015982374
the car plant. 0.10728803181232427
the garden rose. 0.38502770576215517
the sheep dog. 0.43958279217854346
the coin dollar. 0.1950694853666537
the library shelf. 0.15681434104503206
the baby mattress. 0.2176465606786666
the plant house. 0.30632

the ant hill. 0.37484532579184604
the pet kitten. 0.24636829272930494
the house pool. 0.32508738674122317
the dog face. 0.1415778771661221
the skin cap. 0.4325379325409948
the tiger kitten. 0.05287291454176257
the cow blood. 0.08267763580734257
the bed water. 0.09980756618571818
the fire ball. 0.10098552730649135
the car nut. 0.3626790375784017
the house gun. 0.3789113085215261
the bag body. 0.11028498435436596
the summer hurricane. 0.06998349407708246
the nut milk. 0.23766662892459856
the peach juice. 0.36496517659587546
the potato pie. 0.07276356363105418
the shepherd boy. 0.1563586597224803
the forest fire. 0.5
the pill bottle. 0.3494273676059707
the strawberry plant. 0.11223636361138045
the leg jeans. 0.15633291602471033
the pet owl. 0.10175755485472071
the police dog. 0.5
the kitchen door. 0.49308994423714503
the bird seat. 0.21495804097901383
the school library. 0.5
the chimp mother. 0.0716252807389095
the lizard tongue. 0.1085381388718516
the milk door. 0.10447852027359017
the c

the toilet lid. 0.1248750591112564
the farmer corn. 0.05178578842251613
the school clown. 0.41519240099951543
the picture menu. 0.4173488678080721
the school bible. 0.15131424797638915
the police telephone. 0.48650778545431694
the jeans dress. 0.12533321622658666
the beach picture. 0.4260702513525765
the baby bottle. 0.18407042689265252
the school newspaper. 0.1586673533751288
the forest mountain. 0.1320052595873208
the pants leg. 0.32554956332139395
the bomb car. 0.35826272769348355
the orchestra money. 0.14323946778970187
the nurse school. 0.4978976557696727
the corn farmer. 0.3458539927652874
the iris bed. 0.20773787530318166
the home ship. 0.07311447940957372
the tomato soup. 0.1586673533751288
the cigarette money. 0.3837619768624103
the mother plant. 0.12723086514807225
the dog dish. 0.1423728515668441
the brick fire. 0.23958593478122647
the prison door. 0.15842888563499466
the fire bell. 0.4083139805096718
the garden cottage. 0.37773055398491473
the mountain lake. 0.4697726849235

the shark skin. 0.09211004286227975
the summer rain. 0.2960607837071446
the home cottage. 0.4745208860601586
the forest pine. 0.2936196788270069
the army field. 0.49042738889366055
the key arrow. 0.26743383901606355
the fire forest. 0.15835238270668361
the house rabbit. 0.38153085429846456
the candy cane. 0.1586673533751288
the dress shirt. 0.1541273824879688
the school cop. 0.12002462354205091
the goat pet. 0.21138307489976715
the magazine book. 0.06747633169815903
the wife farmer. 0.32949979400476614
the bird holiday. 0.1197393820993845
the baker book. 0.2647345647484226
the dog bull. 0.08085947695362405
the child infant. 0.325817738262802
the field vehicle. 0.2995030812598356
the stage tights. 0.3301728613589674
the smoke cigar. 0.2886434853171944
the river police. 0.11166622535092963
the milk infant. 0.06382209290290941
the holiday summer. 0.08150944252525016
the bubble gum. 0.1586673533751288
the bridge ice. 0.05950395729820157
the mattress baby. 0.0703790150064657
the breast milk

,Stim,GPT Temp 0,GPT Temp .5,GPT Temp 1,GPT Continuous Temp 0,GPT Continuous Temp .5,GPT Continuous Temp 1,Class,Humans Continuous,SD,...,Logw1frq,Logw2frq,Logwordfrq_sum,pairs,Word2Vec,GloVe,Taxonomic,Humans Binary,Difference Score,Distribution
0,the dog puppy.,Makes Sense,Makes Sense,Makes Sense,4,4,4,-1,1.86,1.19,...,3.11,1.94,5.05,dog puppy,0.810643,0.593606,0.787731,Ambiguous,2.14,"[1.0, 1.0, 4.0, 1.0, 3.0, 1.0, 2.0, 2.0, 2.0, ..."
1,the baby teeth.,Makes Sense,Makes Sense,Makes Sense,2,2,2,1,3.82,0.48,...,3.52,3.14,6.66,baby teeth,0.202178,0.313774,0.375739,Makes Sense,1.82,"[4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, ..."
2,the dress summer.,Makes Sense,Makes Sense,Makes Sense,2,2,2,-1,1.00,1.24,...,3.18,3.34,6.52,dress summer,0.094827,0.213653,0.126392,Nonsense,1.00,"[1.0, 0.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, ..."
3,the boot car.,Nonsense,Nonsense,Nonsense,2,1,2,-1,0.71,0.98,...,2.20,3.69,5.90,boot car,0.223294,0.163418,0.444087,Nonsense,1.29,"[0.0, 0.0, 2.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, ..."
4,the pig truck.,Nonsense,Nonsense,Nonsense,2,2,2,1,1.93,1.56,...,2.51,2.65,5.15,pig truck,0.189152,0.151660,0.311380,Ambiguous,0.07,"[3.0, 3.0, 2.0, 2.0, 2.0, 1.0, 3.0, 4.0, 1.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,the goat cheese.,Makes Sense,Makes Sense,Makes Sense,3,2,3,1,3.83,0.76,...,2.32,2.70,5.02,goat cheese,0.317911,0.606518,0.427718,Makes Sense,0.83,"[4.0, 3.0, 4.0, 4.0, 3.0, 3.0, 3.0, 3.0, 3.0, ..."
2156,the sand beach.,Makes Sense,Makes Sense,Makes Sense,2,2,3,-1,2.68,1.25,...,2.98,3.03,6.00,sand beach,0.520518,0.468190,0.487241,Makes Sense,0.68,"[1.0, 3.0, 3.0, 3.0, 3.0, 3.0, 2.0, 3.0, 2.0, ..."
2157,the mouth pig.,Makes Sense,Makes Sense,Makes Sense,1,1,2,-1,0.29,0.60,...,3.38,2.51,5.89,mouth pig,0.314613,0.395392,0.279621,Nonsense,0.71,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, ..."
2158,the field flower.,Makes Sense,Makes Sense,Makes Sense,2,2,3,-1,2.70,1.46,...,3.37,2.70,6.06,field flower,0.124187,0.124687,0.296407,Makes Sense,0.70,"[3.0, 2.0, 2.0, 1.0, 3.0, 2.0, 2.0, 2.0, 3.0, ..."


In [377]:
remove_middle = all_words_and_measures[all_words_and_measures['Humans Binary'].isin(['Makes Sense', 'Nonsense'])]

for index, row in all_words_and_measures.iterrows():
    print(int(index))
# all_words_and_measures['Distribution'][0]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [137]:
p_value = .05

min_value = 0
max_value = 4
all_pairs_and_pvalues = {}
significant_pairs = {}

remove_middle['Distribution'] = ''

GenerateDistributions(remove_middle)

print('#########')
print(significant_pairs)
print('#############')
print('Number of significant pairs: ', len(significant_pairs))

C:\Users\Nick\AppData\Local\Temp\ipykernel_16980\294921397.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remove_middle['Distribution'] = ''


the dress summer. 0.15879468265858088
the heart guy. 0.10669782845467402
the helmet mirror. 0.3138746018123135
the field square. 0.25942787480197405
the pen pig. 0.17894327235887664
the bike barn. 0.3218801565740246
the flag rainbow. 0.050393080519483546
the juice pickle. 0.09944307821372145
the beach summer. 0.2831685033005724
the dime bag. 0.17408474521938833
the sword cane. 0.1098155548729085
the sky rocket. 0.2524685884544927
the tree snow. 0.2188661327306114
the pig pet. 0.18736674274285992
the leg bird. 0.0909923011837479
the rocket guy. 0.3142122579965059
the door barn. 0.23895025951405247
the oven home. 0.0925752662969982
the toe nail. 0.06990152689261452
the skin banana. 0.07129947351871024
the winter wolf. 0.1628112474398224
the book jacket. 0.21015638854241753
the rock pet. 0.33831043277441575
the clothes police. 0.31573318827329744
the book baby. 0.14464183890979476
the duck meat. 0.08974311871798991
the island sand. 0.07577122284372483
the tomato salad. 0.39060979668764295

the plant magazine. 0.34770276885889584
the hay field. 0.3856948466056269
the cottage cheese. 0.12877324055064124
the sled dog. 0.3136525020718641
the mountain lion. 0.10317036151014805
the mountain cave. 0.07684072339604593
the face sheet. 0.2582407560298754
the onion soup. 0.1586673533751288
the police school. 0.25506145264333774
the ape body. 0.22781918432594894
the ice mountain. 0.4620824889339633
the vehicle radio. 0.4526559878766757
the pool water. 0.35376853307536366
the vehicle engine. 0.4956602344337683
the dog picture. 0.4678930234921217
the midnight cop. 0.40458541123466585
the crib sheet. 0.2572935007765904
the bell tower. 0.1586673533751288
the bread flour. 0.3343720659857732
the typewriter key. 0.4851706239449837
the door key. 0.42384195523836654
the chess book. 0.4268882930031621
the refrigerator door. 0.1586673533751288
the kitchen window. 0.1283233627425258
the baby picture. 0.18243517423956968
the book worm. 0.09193392861386467
the brick home. 0.08127497583975869
the 

the holiday home. 0.07415091092894444
the nut car. 0.05880105489139653
the lemon pepper. 0.05835674523352019
the home television. 0.10641054548669027
the library book. 0.5
the tree house. 0.38087059805660084
the railroad building. 0.052290225016175085
the onion bulb. 0.07046562762162989
the school book. 0.25070288029248644
the cop street. 0.056025827993691046
the fire engine. 0.22029824602510084
the chicken rib. 0.06799153182171436
the knee pants. 0.22509505851722544
the pendulum clock. 0.17683620126113242
the knee cap. 0.1916267900511106
the brick fireplace. 0.37651255655576543
the tennis queen. 0.050194999285902875
the pine forest. 0.4910785614921297
the cap field. 0.06409131655827272
the kitchen dish. 0.08469010571440738
the crab cake. 0.14326999835265686
the infant puppy. 0.21626390172577087
the water truck. 0.0726774721607385
the leg pants. 0.09936323378442813
the summer rose. 0.08676788684637349
the butt bubble. 0.13952667764707039
the summer beach. 0.074194617721562
the boot men

the door library. 0.17969359319047618
the bird sea. 0.07271643846635745
the grape juice. 0.1586673533751288
the diamond ear. 0.16880216312096907
the skin shark. 0.06291520385065774
the tower sky. 0.11391163568979479
the fountain water. 0.4830279945491885
the cab car. 0.1745663307143906
the gun bag. 0.26722817896609397
the window hotel. 0.06250168439748383
the shark skin. 0.08806126237667461
the summer rain. 0.2869948530221151
the forest pine. 0.2898377290911236
the army field. 0.4947780189445299
the key arrow. 0.26921176893607024
the candy cane. 0.1586673533751288
the dress shirt. 0.15946523873172985
the school cop. 0.11425548655596565
the goat pet. 0.2035370885672414
the wife farmer. 0.33057005005129914
the bird holiday. 0.11554042304986578
the baker book. 0.2625389518350458
the dog bull. 0.07999669464335579
the child infant. 0.33144173134093613
the field vehicle. 0.3019679790155705
the stage tights. 0.3273789185818058
the smoke cigar. 0.28997144056873503
the milk infant. 0.0602983155

In [352]:
fake_subjects = pd.DataFrame(all_words_and_measures['Stim'])

# def GenFakeData(series_to_be_shuffled, new_df):
#     for i in range(1000):
#         random.shuffle(series_to_be_shuffled)
#         new_df[i] = series_to_be_shuffled.str[0]   

# GenFakeData(all_words_and_measures['Distribution'], fake_subjects)

fake_subjects['test sub 1'] = ''

# for j in range(10,100):
#     for i in range(len(fake_subjects)):
#         fake_subjects[j][i] = random.choice(all_words_and_measures_short['Distribution'][2])

for j in range(0,10000):
    for i in range(len(all_words_and_measures)):
        fake_subjects.at[i, j] = random.choice(all_words_and_measures['Distribution'][i])
#         print(row[0])
        

fake_subjects['Humans Continuous'] = all_words_and_measures['Humans Continuous']
fake_subjects['SD'] = all_words_and_measures['SD']    
fake_subjects

C:\Users\Nick\AppData\Roaming\Python\Python39\site-packages\pandas\core\indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)
C:\Users\Nick\AppData\Local\Temp\ipykernel_16980\1493075452.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  fake_subjects['Humans Continuous'] = all_words_and_measures['Humans Continuous']
C:\Users\Nick\AppData\Local\Temp\ipykernel_16980\1493075452.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fra

,Stim,test sub 1,0,1,2,3,4,5,6,7,...,9992,9993,9994,9995,9996,9997,9998,9999,Humans Continuous,SD
0,the dog puppy.,,1.0,2.0,2.0,3.0,3.0,1.0,2.0,2.0,...,2.0,3.0,1.0,2.0,1.0,2.0,1.0,3.0,1.86,1.19
1,the baby teeth.,,4.0,4.0,4.0,4.0,4.0,4.0,3.0,4.0,...,4.0,4.0,3.0,4.0,3.0,4.0,4.0,3.0,3.82,0.48
2,the dress summer.,,2.0,1.0,1.0,0.0,2.0,1.0,1.0,2.0,...,3.0,1.0,3.0,2.0,2.0,2.0,1.0,1.0,1.00,1.24
3,the boot car.,,1.0,2.0,2.0,0.0,1.0,0.0,0.0,0.0,...,2.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.71,0.98
4,the pig truck.,,2.0,1.0,4.0,2.0,3.0,2.0,2.0,1.0,...,4.0,2.0,2.0,0.0,2.0,3.0,1.0,2.0,1.93,1.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,the goat cheese.,,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,3.0,3.0,3.0,3.0,4.0,4.0,4.0,3.83,0.76
2156,the sand beach.,,3.0,1.0,2.0,2.0,1.0,2.0,3.0,2.0,...,3.0,3.0,3.0,3.0,2.0,3.0,3.0,2.0,2.68,1.25
2157,the mouth pig.,,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,...,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.29,0.60
2158,the field flower.,,4.0,2.0,3.0,4.0,2.0,2.0,2.0,3.0,...,3.0,2.0,3.0,3.0,2.0,3.0,1.0,2.0,2.70,1.46


In [ ]:
p_value = .05

min_value = 0
max_value = 4

number_of_significant_tests = []

def TestFakeDistributions(df):
    for i in range(0,10000):
        all_pairs_and_pvalues = {}
        significant_pairs = {}
        for index, row in df.iterrows():
            mean = row['Humans Continuous']
            std = row['SD']
            case = row[i]
            stim = row['Stim']
            distribution_index = all_words_and_measures['Distribution'][int(index)]
        
#             if std == 0:
#                 min_in_standard_domain = (min_value - mean)
#                 max_in_standard_domain = (max_value - mean)
#             else:
#                 min_in_standard_domain = (min_value - mean) / std
#                 max_in_standard_domain = (max_value - mean) / std
    
#             min_in_erf_domain = special.erf(min_in_standard_domain)
#             max_in_erf_domain = special.erf(max_in_standard_domain)
    
#             random_uniform_data = np.random.uniform(min_in_erf_domain, max_in_erf_domain, 10000)
#             random_gaussianized_data = (special.erfinv(random_uniform_data) * std) + mean
    
#             random_gaussianized_data = np.around(random_gaussianized_data,0)
    
            p_obs = CrawfordTest(case, distribution_index)
        
#             print(stim, CrawfordTest(case, random_gaussianized_data))
#             fig, axes = plt.subplots(1, 1, figsize=(6, 3))
#             axes.hist(random_gaussianized_data, 5, range=(0,4))
#             axes.set_title(stim)
#             plt.axvline(x=case, color='r')
# #     plt.axvline(x=row['GPT Continuous Temp .5'], color='y')
# #     plt.axvline(x=row['GPT Continuous Temp 1'], color='c')
# #    plt.savefig(row[0]+'.png')
#             plt.show()
    
            all_pairs_and_pvalues.update({stim:p_obs})
    
            if p_obs < p_value:
                significant_pairs.update({stim:p_obs})
                continue
            else:
                pass
    
#         print('processing case: ', i)    
#         print(len(significant_pairs))
        number_of_significant_tests.append(len(significant_pairs))
        print(len(number_of_significant_tests))

TestFakeDistributions(fake_subjects)
# print(number_of_significant_tests)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699


5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338


6778
6779
6780
6781
6782
6783
6784
6785
6786
6787
6788
6789
6790
6791
6792
6793
6794
6795
6796
6797
6798
6799
6800
6801
6802
6803
6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
